# Downloading Free Intraday Stock Data from Google Finance

In [21]:
import csv
import datetime
import re
import codecs
import requests

import pandas as pd
import numpy as np

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
init_notebook_mode(connected=True)

In [3]:
cf.go_offline()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [117]:
def get_google_finance_intraday(ticker, period=60, days=1, exchange='NASD'):
    """
    Retrieve intraday stock data from Google Finance.
    
    Parameters
    ----------------
    ticker : str
        Company ticker symbol.
    period : int
        Interval between stock values in seconds.
        i = 60 corresponds to one minute tick data
        i = 86400 corresponds to daily data
    days : int
        Number of days of data to retrieve.
    exchange : str
        Exchange from which the quotes should be fetched
    
    Returns
    ---------------
    df : pandas.DataFrame
        DataFrame containing the opening price, high price, low price,
        closing price, and volume. The index contains the times associated with
        the retrieved price values.
    """

    # build url
    url = 'https://finance.google.com/finance/getprices' + \
          '?p={days}d&f=d,o,h,l,c,v&q={ticker}&i={period}&x={exchange}'.format(ticker=ticker, 
                                                                               period=period, 
                                                                               days=days,
                                                                               exchange=exchange)
    
    page = requests.get(url)
    reader = csv.reader(codecs.iterdecode(page.content.splitlines(), "utf-8"))
    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    rows = []
    times = []
    for row in reader:
        if re.match('^[a\d]', row[0]):
            if row[0].startswith('a'):
                start = datetime.datetime.fromtimestamp(int(row[0][1:]))
                times.append(start)
            else:
                times.append(start+datetime.timedelta(seconds=period*int(row[0])))
            rows.append(map(float, row[1:]))
    if len(rows):
        return pd.DataFrame(rows, index=pd.DatetimeIndex(times, name='Date'), columns=columns)
    else:
        return pd.DataFrame(rows, index=pd.DatetimeIndex(times, name='Date'))

In [118]:
ticker = 'AAPL'
period = 60
days = 1
exchange = 'NASD'

In [119]:
len(df.Close)

215

In [120]:
df = get_google_finance_intraday(ticker, period=period, days=days)

In [121]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-12-04 15:30:00,172.4800,172.4900,172.4400,172.4800,405326.0
2017-12-04 15:31:00,172.3200,172.5000,172.2100,172.4550,207211.0
2017-12-04 15:32:00,172.2500,172.6200,172.2500,172.3000,208746.0
2017-12-04 15:33:00,172.3700,172.4400,172.1300,172.2500,148125.0
2017-12-04 15:34:00,171.8800,172.4300,171.8800,172.3684,186284.0
2017-12-04 15:35:00,172.0600,172.1000,171.7600,171.8600,140023.0
2017-12-04 15:36:00,171.7100,172.0600,171.6900,172.0600,162730.0
2017-12-04 15:37:00,171.7800,171.8000,171.6021,171.7000,159126.0
2017-12-04 15:38:00,172.0400,172.0400,171.7899,171.8000,110316.0


## Plotting

In [115]:
df[['Open', 'High', 'Low', 'Close']].iplot(kind='candle', up_color='#9900cc', down_color='#00ffcc', theme='solar', 
                                           title='Candlesticks for Intraday Prices of {ticker}'.format(ticker=ticker), xTitle='Time')

In [94]:
df.to_csv('./data/{ticker}.csv'.format(ticker=ticker), sep=';', header=False, index=False)